In [1]:
###### Config #####
import sys, os, platform
if os.path.isdir("ds-assets"):
  !cd ds-assets && git pull
else:
  !git clone https://github.com/lutzhamel/ds-assets.git
colab = True if 'google.colab' in os.sys.modules else False
system = platform.system() # "Windows", "Linux", "Darwin"
home = "ds-assets/assets/"
sys.path.append(home)  

Already up to date.


In [2]:
# notebook level imports
import pandas as pd
import numpy as np
import dsutils
np.set_printoptions(formatter={'float_kind':"{:3.2f}".format})
from sklearn import tree
from sklearn import model_selection
from sklearn import metrics

import pandas as pd


# Model Building and Uncertainty

Building models carries with it a **certain amount of uncertainty**.
Recall that machine learning is an inductive activity: We learn from examples and try to generalize into patterns/hypotheses/theories. In general, our examples are datasets that represent a **sample** from a much
larger domain.  Recall the "black swan problem" where the overall domain of swans contains both white 
and black swans.  But the white swans outnumber the black swans by a substantial margin and therefore most samples "D" drawn from
the overall population "X" will only contain white swans as can be seen in the figure below,

<center>
<img 
  src="https://raw.githubusercontent.com/lutzhamel/ds-assets/main/assets/black-swans.png"  
  height="200" 
  width="250">
</center>

This means, if we learn from the samples we will come to the conclusion that "all swans are white".

What this example illustrates is that the quality of our model is very much dependent on the quality 
of the learning data.  Unfortunately, in most cases the machine learning practitioner has no control over
the construction of the data sample. For instance, in a bank application it is simply the database of all the customers of that particular bank which represents a sample of all bank customers of all banks worldwide. 

This sample representation of the domain is one source of uncertaintly when building models.  Another source of ucertaintly is the construction of training and testing sets from the sample.  In the previous notebook we discussed techniques that minimize the impact of those splits but a certain amount of uncertainty always remains.

Consider the following examples that use the iris dataset.


In [3]:
df = pd.read_csv(home+"iris.csv")
X  = df.drop(['id','Species'],axis=1)
y = df['Species']

Using train-test splits to build models,

In [4]:
for i in range(5):
   model = tree.DecisionTreeClassifier(criterion='entropy', max_depth=2)
   (X_train, X_test, y_train, y_test) = \
      model_selection.train_test_split(X, 
                                       y, 
                                       train_size=0.7, 
                                       test_size=0.3)
   model.fit(X_train, y_train)
   y_test_model = model.predict(X_test)
   print("Accuracy {}: {:3.2f}"
         .format(i,metrics.accuracy_score(y_test, y_test_model)))

Accuracy 0: 0.98
Accuracy 1: 0.91
Accuracy 2: 0.89
Accuracy 3: 0.96
Accuracy 4: 0.89


We can see that the way we split has a huge impact on the model quality.  But we knew that already and therefore we used the cross-validation approach,

In [5]:
for i in range(5):
   model = tree.DecisionTreeClassifier(criterion='entropy', max_depth=2)
   mean_score = model_selection.cross_val_score(model, 
                                                X, 
                                                y, 
                                                cv=model_selection.KFold(n_splits=5,
                                                                         shuffle=True)).mean()
   print("Mean Accuracy {}: {:3.2f}".format(i, mean_score))

Mean Accuracy 0: 0.91
Mean Accuracy 1: 0.95
Mean Accuracy 2: 0.94
Mean Accuracy 3: 0.92
Mean Accuracy 4: 0.92


However, if with the cross-validation approach we still see a substantial variation in the observed accuracies, perhaps not as drastic as in the test-split approach.



This uncertainty reflects into our model evaluation. If our training data is a poor representation of the data universe then the models we construct using it will generalize poorly to the rest of the data universe. If our training data is a good representation of the data universe then we can expect that our model will generalize well.


# Classification Confidence Intervals

We use **confidence intervals** in order to quantify the uncertainties discussed above in our analyses.

First, let us define confidence intervals formally. Given a model accuracy, *acc*, then the confidence interval is defined as the probability *p* that our model accuracy *acc* lies between some lower bound *lb* and some upper bound *ub*,

$$
Pr(lb \le acc \le ub) = p.
$$

Paraphrasing this equation with *p = 95%*:

> We are 95% percent sure that our model accuracy is not worse than *lb* and not better than *ub*.


Ultimitely we are interested in the lower and upper bounds of the 95% confidence interval.  We can use the following formula to compute the bounds:

$$ub = acc + 1.96 \sqrt \frac{acc (1 - acc)}{n}$$

$$lb = acc - 1.96 \sqrt \frac{acc (1 - acc)}{n}$$

Here, *n* is the number of observations in the testing dataset used to estimate *acc*. The constant 1.96 is called the *z-score* and expresses the fact that we are computing the 95% confidence interval.


Let's do a simple example using the function `classification_confint`.

In [6]:
# assume the following
observations = 100
acc = .88

# then
lb,ub = dsutils.classification_confint(acc,observations)
print('Accuracy with confidence interval: {} ({:3.2f},{:3.2f})'.format(acc,lb, ub))

Accuracy with confidence interval: 0.88 (0.82,0.94)


Now, let's do an actual example using our iris dataset.  We want to print out the testing accuracy together with it's 95% confidence interval. We construct a model with limited complexity and train and test it

In [7]:
model = tree.DecisionTreeClassifier(criterion='entropy', max_depth=2)
(X_train, X_test, y_train, y_test) = \
    model_selection.train_test_split(X, 
                                    y, 
                                    train_size=0.7, 
                                    test_size=0.3)
model.fit(X_train, y_train)
y_test_model = model.predict(X_test)

# compute the accuracy of optimal classifier
acc = metrics.accuracy_score(y_test, y_test_model)
observations = X_test.shape[0]
lb,ub = dsutils.classification_confint(acc,observations)

# print accuracy
print("Accuracy with confidence interval: {:3.2f} ({:3.2f}, {:3.2f})".format(acc, lb, ub))

Accuracy with confidence interval: 0.96 (0.90, 1.00)


# Regression Confidence Intervals

When performing regression we use the $R^2$ score to examine the quality of our models.  Given that we only use a small training dataset for fitting the model compared to the rest of the data universe it is only natural to ask what the 95% confidence interval for this score might be.  We have a formula for that -- it is not as straight forward as the confidence interval for classification,

$$lb = R^2 - 2\sqrt{\frac{4R^{2}(1-R^{2})^{2}(n-k-1)^{2}}{(n^2 - 1)(n+3)}}$$

$$ub = R^2 + 2\sqrt{\frac{4R^{2}(1-R^{2})^{2}(n-k-1)^{2}}{(n^2 - 1)(n+3)}}$$

Here, *n* is the number of observations in the validation/testing dataset and *k* is the number of independent variables.

In [8]:
# assume the following
rs_score = .75
observations = 100
variables = 4 # independent variables

# then
lb,ub = dsutils.regression_confint(rs_score, observations, variables)
print("R^2 Score with confidence interval: {:3.2f} ({:3.2f}, {:3.2f})".format(rs_score,lb,ub))

R^2 Score with confidence interval: 0.75 (0.67, 0.83)


Let's look at an actual regression problem and compute the $R^2$ score and it's 95% confidence interval. We will use the cars problem from before.

In [9]:
# get our dataset
cars_df = pd.read_csv(home+"cars.csv")

# build model object
model = tree.DecisionTreeRegressor()

param_grid = {
    'max_depth': list(range(1,11)), # search 1..10
    }
grid = model_selection.GridSearchCV(model, param_grid, cv=5)

# performing grid search
grid.fit(cars_df[['speed']],cars_df[['dist']])

# R^2 score
rs_score = grid.best_estimator_.score(cars_df[['speed']],cars_df[['dist']])
observations = cars_df.shape[0]
variables = 1
lb,ub = dsutils.regression_confint(rs_score, observations, variables)

# print out R^2 score with its 95% confidence interval
print("R^2 Score with confidence interval: {:3.2f} ({:3.2f}, {:3.2f})".format(rs_score,lb,ub))

R^2 Score with confidence interval: 0.76 (0.66, 0.87)


# Statistical Significance

Besides giving us an idea of the uncertainty of our model the 95% confidence intervals also have something to say about the significance of scores of different models.  That is, if the confidence intervals overlap then the difference in model performance of two different models on the same dataset is not statistically significant.

Consider the following,

In [10]:
observations = 100

# assume first classifier
acc1 = .88
lb1,ub1 = dsutils.classification_confint(acc1,observations)
print('Accuracy: {} ({:3.2f},{:3.2f})'.format(acc1,lb1, ub1))

# assume second classifier
acc2 = .92
lb2,ub2 = dsutils.classification_confint(acc2,observations)
print('Accuracy: {} ({:3.2f},{:3.2f})'.format(acc2,lb2, ub2))

Accuracy: 0.88 (0.82,0.94)
Accuracy: 0.92 (0.87,0.97)


Even though the second classifier has a better raw accuracy when we look at the confidence intervals of the two classifiers we see that they overlap.  Here we see that the first classifier could potentially have an accuracy of .94 (even better than the raw accuracy of the second classifier).  Furthermore, the confidence interval of the second classifier tells us that that classifier could potentially have an accuracy of .87 which is worse than the raw accuracy of the first classifier.  For this reason we say that the difference in accuracy of two classifiers is not statistically significant if their confidence intervals overlap.

## A Worked Example

First we will construct the optimal model for the abalone data set which is the same code as we have seen above and then we construct a minimal tree for the same data set and compare the performances using statistical significance.

The optimal tree first.

In [11]:
# Grid search with cross-validation for abalone dataset
import pandas as pd
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

# get data
df = pd.read_csv(home+"abalone.csv")
X  = df.drop(columns=['sex'])
y = df[['sex']]

# setting up grid search
model = tree.DecisionTreeClassifier(random_state=1)
param_grid = {
    'max_depth': list(range(1,11)), # search 1..10
    'criterion': ['entropy', 'gini', 'log_loss']
    }
grid = GridSearchCV(model, param_grid, cv=5)

# performing grid search
grid.fit(X,y)

# compute the accuracy of optimal classifier
predict_y = grid.best_estimator_.predict(X)
acc = accuracy_score(y, predict_y)
observations = X.shape[0]
lb,ub = dsutils.classification_confint(acc,observations)

# print accuracy
print("Accuracy of optimal classifier with confidence interval: {:3.2f} ({:3.2f}, {:3.2f})".format(acc, lb, ub))

Accuracy of optimal classifier with confidence interval: 0.59 (0.57, 0.60)


Now we construct the minimal tree.

In [12]:
# create our model object
model = tree.DecisionTreeClassifier(criterion='entropy', max_depth=2, random_state=1)

# train
model.fit(X,y)

# compute the accuracy of minimal classifier
predict_y = model.predict(X)
acc = accuracy_score(y, predict_y)
observations = X.shape[0]
lb,ub = dsutils.classification_confint(acc,observations)

# print accuracy
print("Accuracy of minimal classifier with confidence interval: {:3.2f} ({:3.2f}, {:3.2f})".format(acc, lb, ub))

Accuracy of minimal classifier with confidence interval: 0.53 (0.52, 0.55)


**Observation**: The confidence intervals are not overlapping, therefore the performance difference is statistically significant!

# Project

Please see BrightSpace.